# Definiciones

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from deap import base, creator, tools, algorithms
from pyproj import Proj
from geopy.distance import geodesic

# Definir las coordenadas de las ciudades
cities = {
    "Seattle": (47.608013, -122.335167),
    "Boise": (43.616616, -116.200886),
    "Everett": (47.967306, -122.201399),
    "Pendleton": (45.672075, -118.788597),
    "Biggs": (45.669846, -120.832841),
    "Portland": (45.520247, -122.674194),
    "Twin Falls": (42.570446, -114.460255),
    "Bend": (44.058173, -121.315310),
    "Spokane": (47.657193, -117.423510),
    "Grant Pass": (42.441561, -123.339336),
    "Burns": (43.586126, -119.054413),
    "Eugene": (44.050505, -123.095051),
    "Lakeview": (42.188772, -120.345792),
    "Missoula": (46.870105, -113.995267)
}

# Proyección cartográfica
projection = Proj(proj='aeqd', ellps='WGS84', datum='WGS84', lat_0=0, lon_0=0)

def project_coordinates(lon, lat):
    x, y = projection(lon, lat)
    return x, y

# Proyectar las coordenadas de las ciudades
projected_cities = {city: project_coordinates(lon, lat) for city, (lat, lon) in cities.items()}

# Función de Fitness y Configuraión del Algoritmo Genético

In [2]:
# Convertir índices a nombres de ciudades
def indices_to_cities(indices):
    return [list(cities.keys())[i] for i in indices]

def calculate_total_distance(indices):
    route = indices_to_cities(indices)
    total_distance = 0
    for i in range(len(route)):
        city_a = route[i]
        city_b = route[(i + 1) % len(route)]
        total_distance += geodesic(cities[city_a], cities[city_b]).kilometers
    return total_distance,


# Crear el objeto fitness y los individuos
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)


toolbox = base.Toolbox()

# Función para crear una ruta aleatoria
toolbox.register("indices", random.sample, range(len(cities)), len(cities))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Función de fitness
toolbox.register("evaluate", calculate_total_distance)

# Operadores genéticos
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/len(cities))

# Algoritmo Genético